In [2]:
# Mini projeto de RAG para metadados SAP (DDL / ERD) focado em tabelas QMEL, MARA, KNA1.

from collections import Counter
import math
from typing import List, Dict, Any, Tuple

In [3]:
# =========================
# 1. METADADOS FAKE SAP
# =========================

# Tabelas SAP (fake, mas inspirado na realidade)
SAP_TABLES = [
    {
        "table_name": "QMEL",
        "description": "Notificações de manutenção (falhas, incidentes, ordens de serviço).",
        "business_context": "Tabela central de notificações de manutenção no SAP PM.",
        "primary_key": ["QMNUM"],
        "columns": [
            {"name": "QMNUM", "type": "CHAR(12)", "description": "Número da notificação (chave primária)."},
            {"name": "MATNR", "type": "CHAR(18)", "description": "Código do material/equipamento relacionado."},
            {"name": "KUNNR", "type": "CHAR(10)", "description": "Código do cliente associado à notificação."},
            {"name": "AUFNR", "type": "CHAR(12)", "description": "Número da ordem de manutenção relacionada."},
            {"name": "QMTXT", "type": "CHAR(40)", "description": "Texto curto descrevendo a falha."},
            {"name": "ERDAT", "type": "DATS", "description": "Data de criação da notificação."},
            {"name": "ERNAM", "type": "CHAR(12)", "description": "Usuário que criou a notificação."},
        ],
    },
    {
        "table_name": "MARA",
        "description": "Tabela de materiais, contendo dados básicos de produtos/equipamentos.",
        "business_context": "Base para todos os materiais em processos de vendas, compras e manutenção.",
        "primary_key": ["MATNR"],
        "columns": [
            {"name": "MATNR", "type": "CHAR(18)", "description": "Código do material (chave primária)."},
            {"name": "MTART", "type": "CHAR(4)", "description": "Tipo de material."},
            {"name": "MATKL", "type": "CHAR(9)", "description": "Grupo de mercadorias."},
            {"name": "MEINS", "type": "CHAR(3)", "description": "Unidade de medida base."},
            {"name": "ERNAM", "type": "CHAR(12)", "description": "Usuário que criou o material."},
            {"name": "ERSDA", "type": "DATS", "description": "Data de criação do material."},
        ],
    },
    {
        "table_name": "KNA1",
        "description": "Dados mestres de clientes (nível geral).",
        "business_context": "Contém informações cadastrais de clientes usados em vendas, faturamento, etc.",
        "primary_key": ["KUNNR"],
        "columns": [
            {"name": "KUNNR", "type": "CHAR(10)", "description": "Número do cliente (chave primária)."},
            {"name": "NAME1", "type": "CHAR(35)", "description": "Nome do cliente."},
            {"name": "ORT01", "type": "CHAR(35)", "description": "Cidade do cliente."},
            {"name": "LAND1", "type": "CHAR(3)", "description": "País do cliente."},
            {"name": "REGIO", "type": "CHAR(3)", "description": "Região/estado do cliente."},
        ],
    },
]

In [4]:
# Relações entre as tabelas (EMULA um ERD)
SAP_RELATIONSHIPS = [
    {
        "from_table": "QMEL",
        "to_table": "MARA",
        "from_column": "MATNR",
        "to_column": "MATNR",
        "description": "Relaciona a notificação de manutenção ao material/equipamento afetado.",
        "usage_example": "Obter descrição e atributos técnicos do material relacionado a uma falha.",
    },
    {
        "from_table": "QMEL",
        "to_table": "KNA1",
        "from_column": "KUNNR",
        "to_column": "KUNNR",
        "description": "Relaciona a notificação de manutenção ao cliente associado.",
        "usage_example": "Analisar falhas por cliente, região ou país.",
    },
]

In [5]:
# =========================
# 2. GERAÇÃO DE BLOCOS DE TEXTO
# =========================

def build_table_block(table: Dict[str, Any]) -> str:
    """Gera um texto descritivo de uma tabela SAP."""
    lines = []
    lines.append(f"[TABELA] {table['table_name']}")
    lines.append("")
    lines.append(f"Descrição técnica/funcional: {table['description']}")
    lines.append(f"Contexto de negócio: {table['business_context']}")
    lines.append("")

    if table.get("primary_key"):
        pk_cols = ", ".join(table["primary_key"])
        lines.append(f"Chave primária: {pk_cols}")

    lines.append("")
    lines.append("Principais colunas:")
    for col in table["columns"]:
        lines.append(f"- {col['name']} ({col['type']}): {col['description']}")

    return "\n".join(lines)


def build_relationship_block(rel: Dict[str, Any]) -> str:
    """Gera um texto descritivo de uma relação entre tabelas."""
    lines = []
    lines.append(f"[RELAÇÃO] {rel['from_table']} ↔ {rel['to_table']}")
    lines.append("")
    lines.append(f"Descrição: {rel['description']}")
    lines.append("")
    lines.append("Detalhes técnicos:")
    lines.append(
        f"- JOIN recomendado: {rel['from_table']}.{rel['from_column']} = "
        f"{rel['to_table']}.{rel['to_column']}"
    )
    lines.append(f"- Exemplo de uso: {rel['usage_example']}")
    lines.append("")
    lines.append("Exemplo de SQL aproximado:")
    lines.append(
        f"SELECT *\n"
        f"FROM {rel['from_table']} f\n"
        f"LEFT JOIN {rel['to_table']} t\n"
        f"  ON f.{rel['from_column']} = t.{rel['to_column']};"
    )

    return "\n".join(lines)


def build_all_blocks() -> List[Dict[str, Any]]:
    """
    Gera todos os blocos de texto (tabelas + relações).

    Retorna lista de dicts:
    {
        "id": str,
        "type": "table" | "relationship",
        "text": str
    }
    """
    blocks = []

    for t in SAP_TABLES:
        blocks.append(
            {
                "id": f"TABLE_{t['table_name']}",
                "type": "table",
                "text": build_table_block(t),
            }
        )

    for r in SAP_RELATIONSHIPS:
        rel_id = f"REL_{r['from_table']}_{r['to_table']}"
        blocks.append(
            {
                "id": rel_id,
                "type": "relationship",
                "text": build_relationship_block(r),
            }
        )

    return blocks

In [6]:
# =========================
# 3. "EMBEDDINGS" SIMPLES (BAG OF WORDS)
# =========================

def tokenize(text: str) -> List[str]:
    """Tokenizador simplificado: lower + split por caracteres não alfanuméricos."""
    text = text.lower()
    tokens = []
    current = []
    for ch in text:
        if ch.isalnum():
            current.append(ch)
        else:
            if current:
                tokens.append("".join(current))
                current = []
    if current:
        tokens.append("".join(current))
    return tokens


def embed_text(text: str) -> Dict[str, float]:
    """
    Cria um "vetor" bag-of-words como dict {token: peso},
    usando apenas frequência simples (TF).
    """
    tokens = tokenize(text)
    counts = Counter(tokens)
    # Normaliza por comprimento
    total = sum(counts.values())
    if total == 0:
        return {}
    return {tok: cnt / total for tok, cnt in counts.items()}


def cosine_similarity(vec1: Dict[str, float], vec2: Dict[str, float]) -> float:
    """Calcula similaridade do cosseno entre dois vetores esparsos."""
    # produto escalar
    common_keys = set(vec1.keys()) & set(vec2.keys())
    dot = sum(vec1[k] * vec2[k] for k in common_keys)

    # norma
    norm1 = math.sqrt(sum(v * v for v in vec1.values()))
    norm2 = math.sqrt(sum(v * v for v in vec2.values()))

    if norm1 == 0 or norm2 == 0:
        return 0.0

    return dot / (norm1 * norm2)

In [7]:
# =========================
# 4. "VECTOR STORE" EM MEMÓRIA
# =========================

class SimpleVectorStore:
    """
    Vector store simples em memória, apenas para demo.
    Guarda embeddings de blocos de texto e faz busca de similaridade.
    """

    def __init__(self):
        self.items: List[Dict[str, Any]] = []

    def add_item(self, item_id: str, metadata: Dict[str, Any], text: str):
        emb = embed_text(text)
        self.items.append(
            {
                "id": item_id,
                "metadata": metadata,
                "text": text,
                "embedding": emb,
            }
        )

    def search(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        q_emb = embed_text(query)
        scored: List[Tuple[float, Dict[str, Any]]] = []

        for item in self.items:
            sim = cosine_similarity(q_emb, item["embedding"])
            scored.append((sim, item))

        # Ordena por similaridade decrescente
        scored.sort(key=lambda x: x[0], reverse=True)

        # Retorna apenas itens com sim > 0 e top_k
        results = [it for sim, it in scored if sim > 0][:top_k]
        return results

In [8]:
# =========================
# 5. "LLM FAKE" + RAG
# =========================

def llm_fake(prompt: str) -> str:
    """
    LLM fake apenas para demo.

    Em um cenário real, aqui você chamaria:
    - OpenAI
    - AWS Bedrock (Claude, Llama)
    - Databricks Model Serving
    etc.
    """
    # Neste modo demo, vamos só devolver uma string explicando
    # que este é o local de integração com LLM.
    return (
        ">>> [RESPOSTA GERADA POR LLM AQUI]\n\n"
        "Este é apenas um stub (mock). "
        "Em um ambiente real, o prompt abaixo seria enviado para um modelo de linguagem:\n\n"
        "----- PROMPT ENVIADO AO LLM -----\n"
        f"{prompt}\n"
        "---------------------------------\n"
    )

In [9]:
class SapRagAssistant:
    """
    Assistente RAG para metadados SAP (DDL / ERD).
    """

    def __init__(self, top_k: int = 5):
        self.vector_store = SimpleVectorStore()
        self.top_k = top_k
        self._initialized = False

    def _initialize_if_needed(self):
        if self._initialized:
            return
        blocks = build_all_blocks()
        for blk in blocks:
            self.vector_store.add_item(
                item_id=blk["id"],
                metadata={"type": blk["type"]},
                text=blk["text"],
            )
        self._initialized = True

    def answer_question(self, question: str) -> str:
        """
        Fluxo completo do RAG:
        1. Busca blocos relevantes.
        2. Monta contexto.
        3. Monta prompt.
        4. Chama um "LLM" (fake aqui).
        """
        self._initialize_if_needed()

        results = self.vector_store.search(question, top_k=self.top_k)

        if not results:
            return (
                "Não encontrei blocos relevantes nos metadados para a pergunta:\n"
                f"'{question}'\n"
                "Tente reformular a pergunta ou adicionar mais detalhes."
            )

        # Montar contexto com blocos retornados
        context_parts = []
        for i, item in enumerate(results, start=1):
            header = f"=== BLOCO {i}: {item['id']} (type={item['metadata']['type']}) ==="
            context_parts.append(header)
            context_parts.append(item["text"])
            context_parts.append("")

        context = "\n".join(context_parts)

        # Montar prompt para o LLM
        prompt = f"""
Você é um especialista em SAP, modelagem de dados e engenharia de dados.

Use apenas as informações dos blocos de contexto abaixo para responder
à pergunta do usuário. Se não houver informação suficiente, explique
que não há detalhes suficientes nos metadados.

CONTEÚDO DE CONTEXTO:
{context}

Pergunta do usuário:
{question}

Responda de forma clara, em português, explicando:
- quais tabelas e colunas são relevantes,
- quais joins podem ser feitos,
- e, se fizer sentido, sugira um exemplo de SQL.
"""

        # Chamar LLM (fake nesta demo)
        answer = llm_fake(prompt)
        return answer

In [10]:
# =========================
# 6. EXEMPLOS DE USO
# =========================

def main():
    assistant = SapRagAssistant(top_k=4)

    print("### Exemplo 1: Como relacionar QMEL com cliente e material?\n")
    q1 = "Como posso juntar a tabela QMEL com os dados de cliente e material?"
    ans1 = assistant.answer_question(q1)
    print(ans1)
    print("\n" + "=" * 80 + "\n")

    print("### Exemplo 2: Qual tabela uso para analisar falhas de equipamento ao longo do tempo?\n")
    q2 = "Qual tabela devo usar para analisar falhas de equipamento ao longo do tempo e como conectar isso com clientes?"
    ans2 = assistant.answer_question(q2)
    print(ans2)
    print("\n" + "=" * 80 + "\n")

    print("### Exemplo 3: Pergunta mais genérica sobre SAP PM\n")
    q3 = "Quais são as principais chaves e relacionamentos da QMEL em um cenário de manutenção?"
    ans3 = assistant.answer_question(q3)
    print(ans3)
    print("\n" + "=" * 80 + "\n")


if __name__ == "__main__":
    main()

### Exemplo 1: Como relacionar QMEL com cliente e material?

>>> [RESPOSTA GERADA POR LLM AQUI]

Este é apenas um stub (mock). Em um ambiente real, o prompt abaixo seria enviado para um modelo de linguagem:

----- PROMPT ENVIADO AO LLM -----

Você é um especialista em SAP, modelagem de dados e engenharia de dados.

Use apenas as informações dos blocos de contexto abaixo para responder
à pergunta do usuário. Se não houver informação suficiente, explique
que não há detalhes suficientes nos metadados.

CONTEÚDO DE CONTEXTO:
=== BLOCO 1: TABLE_MARA (type=table) ===
[TABELA] MARA

Descrição técnica/funcional: Tabela de materiais, contendo dados básicos de produtos/equipamentos.
Contexto de negócio: Base para todos os materiais em processos de vendas, compras e manutenção.

Chave primária: MATNR

Principais colunas:
- MATNR (CHAR(18)): Código do material (chave primária).
- MTART (CHAR(4)): Tipo de material.
- MATKL (CHAR(9)): Grupo de mercadorias.
- MEINS (CHAR(3)): Unidade de medida base.
